In [1]:
# core imports
from datetime import datetime
import numpy as np
import pandas as pd
from time import sleep


# Aplhavantage
from alpha_vantage.techindicators import TechIndicators
from alpha_vantage.timeseries import TimeSeries


In [2]:
tickers = ['AAPL','ACN','ADBE','ADI','ADP','ADSK','AKAM','AMAT','AMD','ANET','ANSS','APH','AVGO','BR','CDNS','CDW','CRM','CSCO','CTSH','CTXS','DXC','ENPH','FFIV','FIS','FISV','FLIR','FLT','FTNT','GLW','GPN','HPE','HPQ','IBM','INTC','INTU','IPGP','IT','JKHY','JNPR','KEYS','KLAC','LRCX','MA','MCHP','MPWR','MSFT','MSI','MU','MXIM','NLOK','NOW','NTAP','NVDA','NXPI','ORCL','PAYC','PAYX','PYPL','QCOM','QRVO','SNPS','STX','SWKS','TEL','TER','TRMB','TXN','TYL','V','VRSN','WDC','WU','XLNX','ZBRA']
# tickers = ['AAPL','ACN']

In [10]:
def API(ticker):
    API_key = 'CSMN0LYTQ5UYMVUT'
    API_URL = "https://www.alphavantage.co/query"
    
    
    ts = TimeSeries(key=API_key, output_format='pandas')
    price = ts.get_daily_adjusted(ticker, outputsize='full')[0]
    
    
    
    ti = TechIndicators(key=API_key, output_format='pandas')
    
    macd, meta_data = ti.get_macd(symbol=ticker,interval='daily')
    bbands, meta_data = ti.get_bbands(symbol=ticker,interval='daily')
    rsi,meta_data = ti.get_rsi(symbol=ticker,interval='daily',time_period=15)
    sma5, meta_data = ti.get_sma(symbol=ticker,time_period=5)
    sma15, meta_data = ti.get_sma(symbol=ticker,time_period=15)
    roc, meta_data = ti.get_roc(symbol=ticker,series_type='high',interval='daily')
    cci, meta_data = ti.get_cci(symbol=ticker,time_period=60,interval='daily')
    dx, meta_data = ti.get_dx(symbol=ticker,time_period=60,interval='daily')
    
    data = pd.concat([price, macd, bbands, rsi, sma5, sma15, roc, cci, dx], axis=1)
    data = data.dropna()
    data['ticker'] = ticker
    return data
    
    
    
    
    

In [11]:
def crossover(data):
    data['crossover'] = data['MACD'] > data['MACD_Signal']
    return data
def cutoff(data):
    return data[data.index>=datetime(2015, 1, 1)]
def dropcols(data):
    data = data.drop(columns=['4. close', '7. dividend amount', '8. split coefficient'])
    return data
def feature_names(data):
    data = data.rename(columns={'1. open':'open', '2. high':'high', '3. low':'low', '5. adjusted close':'adjc', '6. volume':'vol'})
    return data

In [5]:
def preprocess_data(tickers):
    for ticker in tickers:
        data = API(ticker)
        data = crossover(data)
        data = cutoff(data)
        data = dropcols(data)
        data.fillna(method='bfill',inplace=True)
        data = feature_names(data)
        data.to_csv('data.csv', mode='a', header=True)
    sleep(7)    
    pass

In [6]:
preprocess_data(tickers)

In [7]:
df = pd.read_csv('data.csv')

In [8]:
df

,date,open,high,low,adjc,vol,MACD_Signal,MACD,MACD_Hist,Real Lower Band,Real Middle Band,Real Upper Band,RSI,SMA,SMA.1,ROC,CCI,DX,ticker,crossover
0,2015-01-02,111.39,111.44,107.35,24.8592220705,53204626.0,0.0331,-0.0331,-0.0662,24.4203,25.4293,26.4382,43.3753,25.4722,25.2811,-4.22,-1.2375,1.023,AAPL,False
1,2015-01-05,108.29,108.65,105.41,24.1588982437,64285491.0,0.0005,-0.13,-0.1305,24.2478,25.3242,26.4006,37.1471,25.1203,25.1997,-7.2952,-39.5183,4.1662,AAPL,False
2,2015-01-06,106.54,107.43,104.63,24.1611720224,65797116.0,-0.0405,-0.2043,-0.1638,24.1051,25.2248,26.3446,37.1785,24.7724,25.1471,-7.4518,-51.0223,5.3946,AAPL,False
3,2015-01-07,107.2,108.2,106.695,24.4999650425,40105934.0,-0.079,-0.2332,-0.1541,24.0206,25.172,26.3233,41.8158,24.5554,25.1399,-5.6242,-32.8844,4.0556,AAPL,False
4,2015-01-08,109.23,112.15,108.7,25.441309407,59364547.0,-0.0988,-0.178,-0.0792,24.0434,25.1466,26.2498,52.2985,24.6241,25.2179,-1.881,14.4123,2.4124,AAPL,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119649,2021-04-19,513.49,518.66,506.79,509.63,177017.0,10.9372,1.6863,9.2509,453.2858,523.5812,488.4335,63.3426,507.236,496.0147,11.4104,105.0502,28.8983,ZBRA,True
119650,2021-04-20,505.39,511.07,496.71,499.56,209338.0,10.062,0.6489,9.4131,456.6722,523.7738,490.223,56.5222,506.236,498.1807,9.1447,82.7821,24.8599,ZBRA,True
119651,2021-04-21,486.81,493.685,483.6938,491.37,349296.0,8.6084,-0.6438,9.2522,460.5846,522.7334,491.659,51.6738,504.38,499.2833,3.0873,51.1202,19.926,ZBRA,False
119652,2021-04-22,496.03,496.03,485.75,487.35,324433.0,7.0507,-1.7612,8.8119,464.3988,521.2732,492.836,49.4433,500.28,499.428,5.7611,50.1916,20.5015,ZBRA,False
